### Cell 1: Check Data Directories
This step remains the same.

In [14]:
!ls /kaggle/input/1000-videos-split/1000_videos/train/fake | head -n 5
!ls /kaggle/input/1000-videos-split/1000_videos/train/real | head -n 5

128_896_10.png
128_896_11.png
128_896_12.png
128_896_13.png
128_896_14.png
ls: write error: Broken pipe
129_10.png
129_2.png
129_3.png
129_4.png
129_5.png
ls: write error: Broken pipe


### Hardware Detection

This cell runs a check to identify the available hardware and selects the appropriate TensorFlow distribution strategy.

In [15]:
import tensorflow as tf
import os

def get_distribution_strategy():
    """
    Detects available hardware (TPU, multi-GPU, single-GPU, CPU) and returns
    the appropriate TensorFlow distribution strategy.
    """
    try:
        # Attempt to detect and initialize a TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        strategy = tf.distribute.TPUStrategy(tpu)
        print("✅ Running on TPU")
    except (ValueError, tf.errors.NotFoundError):
        # If no TPU is found, check for GPUs
        gpus = tf.config.list_physical_devices('GPU')
        if len(gpus) > 1:
            # If multiple GPUs are available, use MirroredStrategy
            strategy = tf.distribute.MirroredStrategy()
            print(f"✅ Running on {len(gpus)} GPUs")
        elif len(gpus) == 1:
            # If a single GPU is available, use the default strategy
            strategy = tf.distribute.get_strategy()
            print("✅ Running on a single GPU")
        else:
            # If no GPUs are found, run on CPU
            strategy = tf.distribute.get_strategy()
            print("✅ Running on CPU")
            
    print(f"Number of accelerator replicas: {strategy.num_replicas_in_sync}")
    return strategy

# Run the detection function to see what hardware is available
strategy = get_distribution_strategy()

✅ Running on a single GPU
Number of accelerator replicas: 1


### Cell 3: Introduction
This cell provides an overview of the notebook's purpose and structure.

# Deepfake Detection with Soft Attention

This notebook sets up and runs the Deepfake Video Forgery Detection project using the original provided Python scripts. The process is broken down into clear, documented stages:

1.  **Setup and Writing Project Files:** Each of the original Python scripts (`model.py`, `utils.py`, `train.py`, `main.py`) will be written to the Kaggle environment. Critical, minimal fixes are applied to ensure the project can run and save the final model.
2.  **Training the Model:** The main training script is executed using a command line call, just as designed in the original project.
3.  **Prediction:** A final script is provided to easily test the trained model on new images.

### Writing `model.py`

This script defines the neural network architecture. The following critical modifications were necessary:

-   **Corrected Layer Instantiation:** Layers are now correctly defined once in the `build()` method and used in the `call()` method. This is essential to allow the model to be saved.
-   **Added `**kwargs`:** The `__init__` methods of the custom layers have been updated to accept extra arguments from Keras during model loading.

In [16]:
%%writefile model.py
import tensorflow as tf

def backbone():
    '''
    RETURNS THE BACKBONE FEATURE ENCODER NETWORK
    XCEPTION USED IN THIS CASE
    '''
    mod  = tf.keras.applications.Xception(weights='imagenet')
    mod = tf.keras.Model(mod.input, mod.layers[-13].output)
    return mod
    
class ModifiedBranch(tf.keras.layers.Layer):
    '''
    COMPUTES THE MODIFIED BRANCH TO BE USED IN ATTENTION TECHNIQUE
    '''
    def __init__(self, a_vec_size, **kwargs):
        super(ModifiedBranch, self).__init__(**kwargs)
        self.a_vec_size = a_vec_size

    def build(self, input_shape):
        self.dense_layer = tf.keras.layers.Dense(self.a_vec_size, activation='tanh')

    def call(self, input):
        af = tf.keras.backend.mean(input, axis=2) 
        hs = self.dense_layer(af)
        return hs

class MainBranch(tf.keras.layers.Layer):
    def __init__(self, a_vec_size, dim, **kwargs):
        super(MainBranch, self).__init__(**kwargs)
        self.a_vec_size = a_vec_size
        self.dim = dim

    def build(self, input_shape):
        self.reshape1 = tf.keras.layers.Reshape((-1, self.a_vec_size))
        self.relu = tf.keras.activations.relu
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.reshape2 = tf.keras.layers.Reshape((self.dim**2, self.a_vec_size))

    def call(self, input):
        e = tf.transpose(input, perm=[0, 2, 1])
        e = self.reshape1(e)
        e = self.relu(e)
        e = self.dropout(e)
        e = self.reshape2(e)
        e = tf.transpose(e, perm=[0, 2, 1])
        return e

class Attention(tf.keras.layers.Layer):
    '''
    IMPLEMENTATION OF THE ATTENTION TECHNIQUE ON TWO BRANCHES
    '''
    def __init__(self, dim, a_vec_size, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.dim = dim
        self.a_vec_size = a_vec_size
    
    def build(self, input_shape):
        self.dense1 = tf.keras.layers.Dense(self.dim**2)
        self.reshape1 = tf.keras.layers.Reshape((1, self.dim**2))
        self.add = tf.keras.layers.Add()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.relu = tf.keras.activations.relu
        self.reshape2 = tf.keras.layers.Reshape((-1, self.a_vec_size))
        self.dense2 = tf.keras.layers.Dense(1, use_bias=False)
        self.reshape3 = tf.keras.layers.Reshape((-1, self.dim**2))

    def call(self, input):
        eh = self.dense1(input[0])
        eh = self.reshape1(eh)
        eh = self.add([input[1], eh])
        eh = self.relu(eh)
        eh = self.dropout(eh)
        eh = tf.transpose(eh, perm=[0, 2, 1])
        eh = self.reshape2(eh)
        eh = self.dense2(eh)
        eh = self.reshape3(eh)
        eh = self.relu(eh)
        return eh

def model(a_vec_size, dim):
    '''
    THIS FUNCTION CALLS THE ENTIRE MODEL
    '''
    back = backbone()
    backbone_feature = back.output  
    out = tf.keras.layers.Conv2D(filters = a_vec_size, kernel_size = (1,1), strides=(1,1), padding = 'valid', use_bias=True)(backbone_feature)
    out = tf.keras.layers.BatchNormalization(axis=-1)(out)
    out = tf.keras.activations.relu(out)
    out = tf.keras.layers.Dropout(0.8)(out)
    out = tf.keras.layers.Reshape((a_vec_size, dim**2))(out)
    
    modified = ModifiedBranch(a_vec_size)(out)
    main = MainBranch(a_vec_size, dim)(out)
    att = Attention(dim, a_vec_size)([modified, main])
    fin = tf.keras.layers.Dense(2, activation='softmax')(att)
    fin = tf.keras.layers.Flatten()(fin)
    mod = tf.keras.Model(inputs=back.input, outputs=fin)
    return mod

Overwriting model.py


### Writing `utils.py`

This script handles data loading. The following modifications were essential:

-   **Corrected Label Reading:** The `get_output` function was changed to read the label from the parent directory name (e.g., `.../fake/` or `.../real/`).
-   **Added Preprocessing:** The `image_generator` now correctly applies `preprocess_input` for the Xception model, which is critical for transfer learning.

In [17]:
%%writefile utils.py
import os
import cv2
import glob
import numpy as np
from tensorflow.keras.applications.xception import preprocess_input

def get_input(path):
    im = cv2.imread(path)
    return(im)

def get_files(path, ext):
    files = []
    label_files= []
    for x in os.walk(path):
        for y in glob.glob(os.path.join(x[0], '*.{}'.format(ext))):
            files.append(y)
    label_files = ['fake', 'real']
    return files, label_files

def get_output(path, label_file):
    img_id = path.split('/')[-2] 
    laba = []
    for label in label_file:
      if label == img_id:
        laba.append(1)
      else:
        laba.append(0)
    return laba

def image_generator(files, label_files, batch_size, resize=None):
    while True:
          batch_paths  = np.random.choice(a  = files, 
                                          size = batch_size)
          batch_x = []
          batch_y = [] 
          
          for input_path in batch_paths:
              input_img = get_input(input_path)
              output = get_output(input_path, label_files)
              
              if resize is not None:
                input_img = cv2.resize(input_img, resize)
              
              input_img = preprocess_input(input_img) 
              
              batch_x.append(input_img)
              batch_y.append(output)

          batch_x = np.array(batch_x)
          batch_y = np.array(batch_y)
          yield batch_x, batch_y

Overwriting utils.py


### Writing `train.py` (Modified for Universal Compatibility)

This script is the main training engine. It has been significantly modified to support all hardware types:

-   **Universal Hardware Detection:** It includes a function that detects TPUs, multiple GPUs, a single GPU, or defaults to CPU.
-   **Strategy Scope:** The model creation and compilation steps are now correctly placed inside the appropriate `strategy.scope()`, which is a requirement for all forms of distributed training (TPU and multi-GPU).
-   **Enabled Model Checkpointing:** The `ModelCheckpoint` callback is enabled to save the best version of the model.

In [18]:
%%writefile train.py
import os
import utils
import model
import tensorflow as tf

def get_distribution_strategy():
    """
    Detects available hardware (TPU, multi-GPU, single-GPU, CPU) and returns
    the appropriate TensorFlow distribution strategy.
    """
    try:
        # Attempt to detect and initialize a TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        strategy = tf.distribute.TPUStrategy(tpu)
        print("✅ Running on TPU")
    except (ValueError, tf.errors.NotFoundError):
        # If no TPU is found, check for GPUs
        gpus = tf.config.list_physical_devices('GPU')
        if len(gpus) > 1:
            # If multiple GPUs are available, use MirroredStrategy
            strategy = tf.distribute.MirroredStrategy()
            print(f"✅ Running on {len(gpus)} GPUs")
        elif len(gpus) == 1:
            # If a single GPU is available, use the default strategy
            strategy = tf.distribute.get_strategy()
            print("✅ Running on a single GPU")
        else:
            # If no GPUs are found, run on CPU
            strategy = tf.distribute.get_strategy()
            print("✅ Running on CPU")
            
    print(f"Number of accelerator replicas: {strategy.num_replicas_in_sync}")
    return strategy

class train():
    def __init__(self, train_path, val_path):
        self.train_path = train_path
        self.val_path = val_path
        here = os.path.dirname(os.path.abspath(__file__))
        self.path = os.path.join(here, "models")
        os.makedirs(self.path, exist_ok=True)
    
    def get_files(self):
        self.train_files, self.label_files = utils.get_files(self.train_path, 'png')
        self.val_files, self.label_files = utils.get_files(self.val_path, 'png')

    def run(self, epochs, batch_size, steps, dim=(299, 299)):
        self.get_files()
        
        # MODIFICATION: Get the appropriate strategy for the available hardware
        strategy = get_distribution_strategy()

        # MODIFICATION: The model must be created and compiled inside the strategy's scope.
        with strategy.scope():
            mod = model.model(1024, 19)
            mod.compile('Adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

        print("************TRAINING SOFT ATTENTION BASED DEEP FAKE DETECTION MODEL************")
        
        checkpoint_filepath = os.path.join(self.path, "best_model.keras")
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath, 
            save_best_only=True, 
            monitor='val_accuracy', 
            mode='max'
        )

        mod.fit(utils.image_generator(self.train_files, self.label_files, batch_size, dim), 
                  epochs=epochs, 
                  steps_per_epoch=steps,
                  validation_data=utils.image_generator(self.val_files, self.label_files, batch_size, dim),
                  validation_steps=50,
                  callbacks=[model_checkpoint_callback])

Overwriting train.py


### Writing `main.py`

This script parses command-line arguments and starts the training. No modifications were needed.

In [19]:
%%writefile main.py
import argparse
from train import train

def main():
    parser = argparse.ArgumentParser(description='Visual Attention based Deepfake Video Forgery Detection')

    parser.add_argument('--train', type=str, nargs = '+', help = 'What is the path of the training image data?')
    parser.add_argument('--val', type=str, nargs = '+', help = 'What is the path of the Validation image data?')
    parser.add_argument('--epochs', type=int, default=50, help = 'What is the training epoch for model?')
    parser.add_argument('--batch', type=int, default=32, help = 'What is the training batch size?')
    parser.add_argument('--steps', type=int, default=40, help = 'What is the training steps per epoch?')

    args = parser.parse_args()
    args.train = ' '.join(args.train)
    args.val = ' '.join(args.val)

    print("Configuration")
    print("----------------------------------------------------------------------")
    print("Training Path : {}".format(args.train))
    print("Validation Path : {}".format(args.val))
    print("Epochs while training the model : {}".format(args.epochs))
    print("Batch Size : {}".format(args.batch))
    print("Steps per epochs : {}".format(args.steps))
    print("----------------------------------------------------------------------")

    train(args.train, args.val).run(args.epochs, args.batch, args.steps)

if __name__=='__main__':
    main()

Overwriting main.py


## Stage 2: Training the Model

Now that all scripts are written and corrected, we can run the training process. 

**Important:** You must update the `--train` and `--val` paths below to point to your dataset's location in the Kaggle input directory (e.g., `/kaggle/input/your-dataset-name/...`). You should also calculate and set the `--steps` argument properly.

**How to calculate `--steps`:**
`steps = (total_number_of_training_images) / batch_size`

In [20]:
# Example Usage: total file no /batch = steps

!python main.py \
    --train "/kaggle/input/1000-videos-split/1000_videos/train" \
    --val "/kaggle/input/1000-videos-split/1000_videos/validation" \
    --epochs 60 \
    --batch 16 \
    --steps 727

2025-09-21 13:22:11.857510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758460931.878537   29099 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758460931.884839   29099 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Configuration
----------------------------------------------------------------------
Training Path : /kaggle/input/1000-videos-split/1000_videos/train
Validation Path : /kaggle/input/1000-videos-split/1000_videos/validation
Epochs while training the model : 60
Batch Size : 16
Steps per epochs : 727
----------------------------------------------------------------------
✅ Running on a single GPU
Number of accelerator replicas: 1
I0000 

In [21]:
!zip -r my_model.zip models/

updating: models/ (stored 0%)
updating: models/best_model.keras (deflated 8%)


## Stage 3: Prediction

After training, a file named `best_model.keras` will be saved in the `/kaggle/working/models/` directory. We can use this model to make predictions on new, unseen images.

First, we'll write a `predict.py` script to handle loading the model and processing an image.

In [22]:
%%writefile predict.py
import tensorflow as tf
import numpy as np
import cv2
import argparse
import os

# --- Import your custom layers and preprocessing function ---
from model import ModifiedBranch, MainBranch, Attention
from tensorflow.keras.applications.xception import preprocess_input

# Define the labels list globally
LABELS = ['fake', 'real']

def load_and_prep_image(image_path, target_size=(299, 299)):
    """
    Loads, resizes, and preprocesses a single image.
    Returns None if the image cannot be read.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read image {image_path}. Skipping.")
            return None
            
        img = cv2.resize(img, target_size)
        img_preprocessed = preprocess_input(img)
        return np.expand_dims(img_preprocessed, axis=0)
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def predict_single_image(model, image_path):
    """
    Loads a single image, predicts it, and prints the result.
    """
    image_batch = load_and_prep_image(image_path)
    if image_batch is None: return

    print("Predicting...")
    prediction = model.predict(image_batch)
    
    predicted_index = np.argmax(prediction[0])
    predicted_label = LABELS[predicted_index]
    confidence = prediction[0][predicted_index] * 100

    print("\n--- Prediction Result ---")
    print(f"       File: {os.path.basename(image_path)}")
    print(f"Prediction is: {predicted_label.upper()}")
    print(f"  Confidence: {confidence:.2f}%")
    print("-------------------------")

def evaluate_folder(model, folder_path):
    """
    Recursively evaluates all images in a given folder.
    """
    print(f"Scanning folder: {folder_path}\nThis may take a while...")
    total_files = 0
    correct_predictions = 0
    
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')): continue
            true_label = os.path.basename(root).lower()
            if true_label not in LABELS: continue

            image_path = os.path.join(root, filename)
            image_batch = load_and_prep_image(image_path)
            if image_batch is None: continue
            
            total_files += 1
            prediction = model.predict(image_batch, verbose=0)
            predicted_index = np.argmax(prediction[0])
            predicted_label = LABELS[predicted_index]

            # --- MODIFICATION IS HERE ---
            confidence = prediction[0][predicted_index] * 100

            if predicted_label == true_label:
                correct_predictions += 1
                result = "CORRECT"
            else:
                result = "WRONG"
            
            # --- AND HERE ---
            print(f"  > File: {filename} | True: {true_label} | Predicted: {predicted_label} ({confidence:.2f}%)  [{result}]")

    if total_files > 0:
        accuracy = (correct_predictions / total_files) * 100
        print("\n--- Evaluation Summary ---")
        print(f"Total Images: {total_files}")
        print(f"Correct Predictions: {correct_predictions}")
        print(f"OVERALL ACCURACY: {accuracy:.2f}%")
        print("--------------------------")
    else:
        print("\nNo valid image files found in 'fake' or 'real' subdirectories.")

def main():
    parser = argparse.ArgumentParser(description='Predict if an image is real or fake.')
    parser.add_argument('--input_path', type=str, required=True, help='Path to an image file OR a folder.')
    parser.add_argument('--model_path', type=str, default='models/best_model.keras', help='Path to the saved model file.')
    args = parser.parse_args()

    if not os.path.exists(args.model_path):
        print(f"Error: Model file not found at {args.model_path}")
        return

    custom_objects = {"ModifiedBranch": ModifiedBranch, "MainBranch": MainBranch, "Attention": Attention}
    print("Loading model...")
    model = tf.keras.models.load_model(args.model_path, custom_objects=custom_objects)
    print("Model loaded.")

    if os.path.isfile(args.input_path):
        predict_single_image(model, args.input_path)
    elif os.path.isdir(args.input_path):
        evaluate_folder(model, args.input_path)
    else:
        print(f"Error: Input path is not a valid file or directory: {args.input_path}")

if __name__ == "__main__":
    main()

Overwriting predict.py


### Running Prediction

Use the command below to test your saved model. You will need to provide a valid path to an image from your dataset.

In [23]:
# Example of how to run prediction on a single image from your validation set.
# You will need to find a valid path to an image.

# !python predict.py --image "/kaggle/input/1000-videos-split/1000_videos/validation/fake/000_003_0.png"
!python predict.py --input_path "/kaggle/input/1000-videos-split/1000_videos/test"

2025-09-21 14:21:28.114446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758464488.136970   29351 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758464488.143910   29351 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading model...
I0000 00:00:1758464492.338465   29351 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
Model loaded.
Scanning folder: /kaggle/input/1000-videos-split/1000_videos/test
This may take a while...
I0000 00:00:1758464499.539748   29375 service.cc:148] XLA service 0x7f8dc80042d0 i